In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot  as plt
import seaborn as sn
%matplotlib inline
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.model_selection import train_test_split


In [2]:
df = pd.read_csv("/home/growlt199/Downloads/manufacturing_project/data/flight_processed_data_wo_stops.csv",index_col=0)

In [3]:
df.head()

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price,travel_route
0,SpiceJet,SG-8709,Delhi,Evening,zero,Night,Mumbai,Economy,2.17,1,5953,Delhi-Mumbai
1,SpiceJet,SG-8157,Delhi,Early_Morning,zero,Morning,Mumbai,Economy,2.33,1,5953,Delhi-Mumbai
2,AirAsia,I5-764,Delhi,Early_Morning,zero,Early_Morning,Mumbai,Economy,2.17,1,5956,Delhi-Mumbai
3,Vistara,UK-995,Delhi,Morning,zero,Afternoon,Mumbai,Economy,2.25,1,5955,Delhi-Mumbai
4,Vistara,UK-963,Delhi,Morning,zero,Morning,Mumbai,Economy,2.33,1,5955,Delhi-Mumbai


In [4]:
#removing colums which i dont want to use in model 
df1= df.drop(['flight', 'travel_route'],axis=1)

In [5]:
df1.head()

,airline,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,SpiceJet,Delhi,Evening,zero,Night,Mumbai,Economy,2.17,1,5953
1,SpiceJet,Delhi,Early_Morning,zero,Morning,Mumbai,Economy,2.33,1,5953
2,AirAsia,Delhi,Early_Morning,zero,Early_Morning,Mumbai,Economy,2.17,1,5956
3,Vistara,Delhi,Morning,zero,Afternoon,Mumbai,Economy,2.25,1,5955
4,Vistara,Delhi,Morning,zero,Morning,Mumbai,Economy,2.33,1,5955


In [6]:
#doing label encoding in class column 
df2 = df1.replace({'class': {'Economy': 0, 'Business': 1}})

# Onehot encoding in all columns because smote oversampling doesn't allow string columns that why i am converting all string column to numeric columns

In [7]:
# get dummy variables for multiple columns
dummy_cols = ['airline', 'departure_time','arrival_time','source_city','destination_city','stops']
df2_dummy = pd.get_dummies(df2[dummy_cols])

# concatenate original DataFrame with dummy variable DataFrame
df3 = pd.concat([df2.drop(dummy_cols, axis=1), df2_dummy], axis=1)

# display result
df3.head()

,class,duration,days_left,price,airline_AirAsia,airline_Air_India,airline_GO_FIRST,airline_Indigo,airline_SpiceJet,airline_Vistara,...,source_city_Mumbai,destination_city_Bangalore,destination_city_Chennai,destination_city_Delhi,destination_city_Hyderabad,destination_city_Kolkata,destination_city_Mumbai,stops_one,stops_two_or_more,stops_zero
0,0,2.17,1,5953,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,1
1,0,2.33,1,5953,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,1
2,0,2.17,1,5956,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
3,0,2.25,1,5955,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,1
4,0,2.33,1,5955,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,1


In [8]:
# X = df3.drop('class' , axis = 1)
# y = df3['class']

In [9]:
#X.head()

In [10]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.over_sampling import SMOTEN
from collections import Counter

def bal_combined_data(X_resampled, y_resampled):
    resampled_data = pd.DataFrame(X_resampled, columns=X.columns)
    resampled_data['class'] = y_resampled
    return resampled_data

def RandomoverS(X,y,k=5):
    
    ros = RandomOverSampler(random_state=0)
    X_resampled, y_resampled = ros.fit_resample(X, y)
    #print(sorted(Counter(y_resampled).items()))
    # Create a new DataFrame with the resampled data
    resampled_data = bal_combined_data(X_resampled, y_resampled)
    return resampled_data
    
def smote(X,y,k=5):    
    X_resampled, y_resampled = SMOTE(k_neighbors=k).fit_resample(X, y)
    
    #print(sorted(Counter(y_resampled).items()))
    
    resampled_data = bal_combined_data(X_resampled, y_resampled)
    return resampled_data
    
def Adasyn(X,y,k=5):
    X_resampled, y_resampled = ADASYN(n_neighbors=k).fit_resample(X, y)
    
    #print(sorted(Counter(y_resampled).items()))
    
    resampled_data = bal_combined_data(X_resampled, y_resampled)
    return resampled_data
    
def Bordarlinesmote(X,y,k=5):
    X_resampled, y_resampled = BorderlineSMOTE(k_neighbors=k).fit_resample(X, y)
    
    #print(sorted(Counter(y_resampled).items()))
    
    resampled_data = bal_combined_data(X_resampled, y_resampled)
    return resampled_data

In [11]:
def regression(X_train,y_train,X_test,y_test):
    import time

    start_time = time.time()

    #print shape of x train,y train, x test and y test 
    print('x_train shape:-',X_train.shape)
    print('x_test shape:-',X_test.shape)
    print('y_train shape:-',y_train.shape)
    print('y_test shape:-',y_test.shape)


    #import model from sklearn
    from sklearn.ensemble import RandomForestRegressor
    print('model name :- random forest regression')

    regr = RandomForestRegressor(random_state=123)

    #training model input as x_train and y_tarin
    regr.fit(X_train,y_train)

    print('score on the testing data :-',regr.score(X_test,y_test))
    print('score on the training data :-',regr.score(X_train,y_train))

    #finding error methods 
    from sklearn import metrics

    #model is creted show give x test data as input and retuen predictd data 
    predictions = regr.predict(X_test)

    #MAE
    print('MAE:-',metrics.mean_absolute_error(y_test,predictions))

    #MSE
    print('MSE:-',metrics.mean_squared_error(y_test,predictions))

    #RMSE
    print('RMSE:-',np.sqrt(metrics.mean_squared_error(y_test,predictions)))

    #r2
    print("r2",r2_score(y_test, predictions))

    # Code or function to measure execution time
    end_time = time.time()
    execution_time = end_time - start_time
    print("Execution time:", execution_time, "seconds")
    
    
    return y_test,predictions

In [12]:
def regression_csv(X_train,y_train,X_test,y_test,item,k):
    import time

    start_time = time.time()

#     #print shape of x train,y train, x test and y test 
#     print('x_train shape:-',X_train.shape)
#     print('x_test shape:-',X_test.shape)
#     print('y_train shape:-',y_train.shape)
#     print('y_test shape:-',y_test.shape)


    #import model from sklearn
    from sklearn.ensemble import RandomForestRegressor
#    print('model name :- random forest regression')

    model = RandomForestRegressor(random_state=123)

    # Fit the model to the training data
    model.fit(X_train, y_train)
    
    # Evaluate the model on the testing data
    y_pred = model.predict(X_test)
    
    # Calculate the mean squared error (MSE)
    mse = mean_squared_error(y_test, y_pred)
    
    #MAE
    mae = metrics.mean_absolute_error(y_test,y_pred)
#     print('MAE:-',metrics.mean_absolute_error(y_test,y_pred))

    #RMSE
    rmse = np.sqrt(metrics.mean_squared_error(y_test,y_pred))

    #r2
    r2 = r2_score(y_test, y_pred)
    
    #mape
    mape =metrics.mean_absolute_percentage_error(y_test, y_pred)
    
    #test and train accurancy
    test_score = model.score(X_test,y_test)
    train_score = model.score(X_train,y_train)

    # Code or function to measure execution time
    end_time = time.time()
    execution_time = end_time - start_time
    #print("Execution time:", execution_time, "seconds")
    
    filename = '/home/growlt199/Downloads/manufacturing_project/model_training/class_oversampling/class_over_sampling_parameters.csv'
  
    with open(filename, 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow([item,k,X_train.shape,X_test.shape,y_train.shape,y_test.shape,
                            mse,mae,mape,rmse,r2,test_score,train_score,execution_time]) 
        
    print(f'{item} and {k} done..')
    
    return y_test,y_pred

In [13]:
def divide_into_train_test(data):
    # Split the data into training and testing sets
    train_data, test_data = train_test_split(data, test_size=0.2, random_state=123,stratify=df['class'])

    # Print the shapes of the train and test sets
    #print("Training set shape:", train_data.shape)
    #print("Testing set shape:", test_data.shape)
    
    return train_data,test_data

In [14]:
def divide_class_and_withoutclass(data):
    X = data.drop('class' , axis = 1)
    y = data['class']
    
    return X,y

In [15]:
def spilt_data(train_data,test_data):
    #create x and y where x are features for the model and y as target
    x_train = train_data.drop('price',axis=1)
    y_train = train_data['price']

    #for test data, create x and y where x are features for the model and y as target
    x_test = test_data.drop('price',axis=1)
    y_test = test_data['price']
    
    return x_train,y_train,x_test,y_test

In [16]:
balancer = ['RandomoverS','smote','Adasyn','Bordarlinesmote']

In [17]:
'''
for item in balancer:
    print(' ')
    print(f'Using {item} for sampling started ..')
    print(' ')
    train_data, test_data = divide_into_train_test(df3)
    X,y = divide_class_and_withoutclass(train_data)
    function = globals().get(item)
    train_data_a = function(X,y,k=5)
    
    # Reorder the DataFrame based on the desired column order
    desired_columns =test_data.columns
    train_data_a = train_data_a.reindex(columns=desired_columns)
    
    X_train,y_train,X_test,y_test = spilt_data(train_data_a,test_data)
    
    y_test,predictions = regression(X_train,y_train,X_test,y_test)
    print(' ')
    print(f'using {item} for sampling done..')
    print(' ')
    
'''

"\nfor item in balancer:\n    print(' ')\n    print(f'Using {item} for sampling started ..')\n    print(' ')\n    train_data, test_data = divide_into_train_test(df3)\n    X,y = divide_class_and_withoutclass(train_data)\n    function = globals().get(item)\n    train_data_a = function(X,y,k=5)\n    \n    # Reorder the DataFrame based on the desired column order\n    desired_columns =test_data.columns\n    train_data_a = train_data_a.reindex(columns=desired_columns)\n    \n    X_train,y_train,X_test,y_test = spilt_data(train_data_a,test_data)\n    \n    y_test,predictions = regression(X_train,y_train,X_test,y_test)\n    print(' ')\n    print(f'using {item} for sampling done..')\n    print(' ')\n    \n"

In [18]:
balancer1 = ['smote','Adasyn','Bordarlinesmote']

In [19]:
import csv 

filename = '/home/growlt199/Downloads/manufacturing_project/model_training/class_oversampling/class_over_sampling_parameters.csv'
    
with open(filename, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['technique','k_neighbors or k_neighbors','X_train_shape','X_test_shape',
                        'y_train_shape','y_test_shape',
                        'mse','mae','mape','rmse','r2','test_score','train_score','execution_time']) 
    
    
#split data 
train_data, test_data = divide_into_train_test(df3)
X,y = divide_class_and_withoutclass(train_data)

counter = Counter(y)
minimum_count = min(counter.values())

for item in balancer1:            
    for k in range(1, 15):
        print(' ')
        print(f'Using {item} for sampling started ..')
        print(' ')
            
        function = globals().get(item)
        train_data_a = function(X,y,k)
            
        # Reorder the DataFrame based on the desired column order
        desired_columns = test_data.columns
        train_data_a = train_data_a.reindex(columns=desired_columns)
    
        X_train,y_train,X_test,y_test = spilt_data(train_data_a,test_data)
    
        y_test,y_pred = regression_csv(X_train,y_train,X_test,y_test,item,k)

 
Using smote for sampling started ..
 
smote and 1 done..
 
Using smote for sampling started ..
 
smote and 2 done..
 
Using smote for sampling started ..
 
smote and 3 done..
 
Using smote for sampling started ..
 
smote and 4 done..
 
Using smote for sampling started ..
 
smote and 5 done..
 
Using smote for sampling started ..
 
smote and 6 done..
 
Using smote for sampling started ..
 
smote and 7 done..
 
Using smote for sampling started ..
 
smote and 8 done..
 
Using smote for sampling started ..
 
smote and 9 done..
 
Using smote for sampling started ..
 
smote and 10 done..
 
Using smote for sampling started ..
 
smote and 11 done..
 
Using smote for sampling started ..
 
smote and 12 done..
 
Using smote for sampling started ..
 
smote and 13 done..
 
Using smote for sampling started ..
 
smote and 14 done..
 
Using Adasyn for sampling started ..
 
Adasyn and 1 done..
 
Using Adasyn for sampling started ..
 
Adasyn and 2 done..
 
Using Adasyn for sampling started ..
 
Adasyn

# analysing results of above 

In [23]:
results = pd.read_csv('/home/growlt199/Downloads/manufacturing_project/model_training/class_oversampling/class_over_sampling_parameters.csv')

In [24]:
results.head()

,technique,k_neighbors or k_neighbors,X_train_shape,X_test_shape,y_train_shape,y_test_shape,mse,mae,mape,rmse,r2,test_score,train_score,execution_time
0,smote,1,"(330654, 36)","(60029, 36)","(330654,)","(60029,)",8.202295e+06,1077.449769,0.069599,2863.964823,0.984020,0.984020,0.997740,83.893511
1,smote,2,"(330654, 36)","(60029, 36)","(330654,)","(60029,)",8.105410e+06,1073.472951,0.069499,2847.000219,0.984208,0.984208,0.997488,85.605268
2,smote,3,"(330654, 36)","(60029, 36)","(330654,)","(60029,)",8.122997e+06,1074.818951,0.069532,2850.087199,0.984174,0.984174,0.997334,87.450081
3,smote,4,"(330654, 36)","(60029, 36)","(330654,)","(60029,)",8.207758e+06,1080.992851,0.069652,2864.918489,0.984009,0.984009,0.997185,88.148164
4,smote,5,"(330654, 36)","(60029, 36)","(330654,)","(60029,)",8.129695e+06,1080.571123,0.069622,2851.262080,0.984161,0.984161,0.997135,88.631997


In [25]:
results.sort_values('test_score', ascending=False)

,technique,k_neighbors or k_neighbors,X_train_shape,X_test_shape,y_train_shape,y_test_shape,mse,mae,mape,rmse,r2,test_score,train_score,execution_time
29,Bordarlinesmote,2,"(330654, 36)","(60029, 36)","(330654,)","(60029,)",7.563961e+06,1073.173719,0.069562,2750.265621,0.985263,0.985263,0.997600,76.759927
28,Bordarlinesmote,1,"(330654, 36)","(60029, 36)","(330654,)","(60029,)",7.566926e+06,1072.968134,0.069559,2750.804663,0.985258,0.985258,0.997605,77.535496
35,Bordarlinesmote,8,"(330654, 36)","(60029, 36)","(330654,)","(60029,)",7.572387e+06,1071.304415,0.069484,2751.797123,0.985247,0.985247,0.997562,84.340391
40,Bordarlinesmote,13,"(330654, 36)","(60029, 36)","(330654,)","(60029,)",7.575285e+06,1070.567816,0.069451,2752.323544,0.985241,0.985241,0.997556,88.299835
36,Bordarlinesmote,9,"(330654, 36)","(60029, 36)","(330654,)","(60029,)",7.577979e+06,1071.907362,0.069499,2752.812869,0.985236,0.985236,0.997560,84.901747
41,Bordarlinesmote,14,"(330654, 36)","(60029, 36)","(330654,)","(60029,)",7.578701e+06,1071.059235,0.069475,2752.944002,0.985235,0.985235,0.997555,84.057046
34,Bordarlinesmote,7,"(330654, 36)","(60029, 36)","(330654,)","(60029,)",7.582754e+06,1070.167805,0.069515,2753.680169,0.985227,0.985227,0.997581,80.783403
30,Bordarlinesmote,3,"(330654, 36)","(60029, 36)","(330654,)","(60029,)",7.585405e+06,1071.354611,0.069520,2754.161307,0.985222,0.985222,0.997594,75.340078
39,Bordarlinesmote,12,"(330654, 36)","(60029, 36)","(330654,)","(60029,)",7.585927e+06,1071.699245,0.069497,2754.256182,0.985221,0.985221,0.997557,93.926116
37,Bordarlinesmote,10,"(330654, 36)","(60029, 36)","(330654,)","(60029,)",7.587203e+06,1072.291543,0.069504,2754.487716,0.985218,0.985218,0.997558,94.958138
